A continuación, te presento el código en Python para que lo uses en Google Colab. Este script tomará cualquier secuencia que le introduzcas, la analizará y generará las tres tablas que necesitas, presentadas con formato de pandas.

### Código Python para Análisis de Secuencias Neumáticas (Método Cascada)

Simplemente copia y pega todo el siguiente bloque de código en una celda de tu Google Colab y ejecútalo.

In [ ]:
import pandas as pd
import re
from IPython.display import display, HTML

def analizar_secuencia_cascada(secuencia: str):
    """
    Analiza una secuencia de movimiento neumática utilizando el método cascada
    y genera tres tablas de resumen en formato pandas.

    Args:
        secuencia (str): La secuencia de movimientos (ej. "A+ B+ B- A-").
                         Los temporizadores se indican con [Xs], ej. "C+[2s]".
    """
    print(f"Análisis para la secuencia: {secuencia}\n")

    # --- 1. Separar la secuencia en movimientos y grupos ---
    # Usamos una expresión regular para encontrar movimientos como A+, B-, C+[2s]
    movimientos = re.findall(r'([A-Z]\+?(\[\d+s\])?-?)', secuencia)
    # Limpiamos la lista para tener solo los strings de movimiento
    movimientos = [m[0] for m in movimientos]

    if not movimientos:
        print("La secuencia introducida no es válida.")
        return

    # Crear grupos: se crea un nuevo grupo cada vez que una letra de pistón se repite
    grupos = []
    grupo_actual = []
    letras_en_grupo = set()

    for mov in movimientos:
        letra = mov[0]
        if letra in letras_en_grupo:
            grupos.append(grupo_actual)
            grupo_actual = [mov]
            letras_en_grupo = {letra}
        else:
            grupo_actual.append(mov)
            letras_en_grupo.add(letra)
    grupos.append(grupo_actual) # Añadir el último grupo

    # --- 2. Asignar finales de carrera ---
    finales_carrera = []
    for mov in movimientos:
        letra = mov[0].lower()
        if '+' in mov:
            finales_carrera.append(f"{letra}1")
        elif '-' in mov:
            finales_carrera.append(f"{letra}0")

    # --- 3. Generar Tabla 1: Desarrollo de la Secuencia ---
    data_desarrollo = {
        'Paso': [],
        'Movimiento': [],
        'Final de Carrera Actuado': [],
        'Grupo': []
    }
    paso_num = 1
    for i, grupo in enumerate(grupos):
        for mov in grupo:
            letra_mov = mov[0].lower()
            signo_mov = '+' if '+' in mov else '-'
            fc_actuado = f"{letra_mov}1" if signo_mov == '+' else f"{letra_mov}0"

            data_desarrollo['Paso'].append(paso_num)
            data_desarrollo['Movimiento'].append(mov)
            data_desarrollo['Final de Carrera Actuado'].append(fc_actuado)
            data_desarrollo['Grupo'].append(f"Grupo {i+1}")
            paso_num += 1

    df_desarrollo = pd.DataFrame(data_desarrollo)

    # --- 4. Generar Tabla 2: Activación de Grupos ---
    num_grupos = len(grupos)
    data_grupos = {'Grupo': [], 'Se Activa Con': []}

    # El último final de carrera de toda la secuencia activa el primer grupo
    ultimo_fc_secuencia = finales_carrera[-1]
    data_grupos['Grupo'].append("Grupo 1")
    data_grupos['Se Activa Con'].append(f"START y Grupo {num_grupos}-{ultimo_fc_secuencia}")

    # Los demás grupos se activan con el último final de carrera del grupo anterior
    for i in range(num_grupos - 1):
        # Encontrar el último movimiento del grupo actual (i) para obtener su FC
        movimientos_grupo_anterior = grupos[i]
        ultimo_mov_grupo_anterior = movimientos_grupo_anterior[-1]
        letra = ultimo_mov_grupo_anterior[0].lower()
        fc = f"{letra}1" if '+' in ultimo_mov_grupo_anterior else f"{letra}0"

        data_grupos['Grupo'].append(f"Grupo {i+2}")
        data_grupos['Se Activa Con'].append(f"Grupo {i+1}-{fc}")

    df_activacion_grupos = pd.DataFrame(data_grupos)

    # --- 5. Generar Tabla 3: Activación de Movimientos ---
    data_movimientos = {} # Usamos un diccionario para agrupar activaciones

    for i, grupo in enumerate(grupos):
        nombre_grupo = f"{i+1}"
        for j, mov in enumerate(grupo):
            if mov not in data_movimientos:
                data_movimientos[mov] = []

            if j == 0: # Si es el primer movimiento del grupo
                # Se activa solo con el relé del grupo
                data_movimientos[mov].append(f"Grupo {nombre_grupo}")
            else: # Si es un movimiento subsiguiente
                # Se activa con el relé del grupo Y el final de carrera anterior
                mov_anterior = grupo[j-1]
                letra_ant = mov_anterior[0].lower()
                fc_anterior = f"{letra_ant}1" if '+' in mov_anterior else f"{letra_ant}0"
                data_movimientos[mov].append(f"Grupo {nombre_grupo}-{fc_anterior}")

    # Formatear el diccionario en un DataFrame
    df_activacion_movs = pd.DataFrame({
        'Movimiento': data_movimientos.keys(),
        'Se Activa Con': [', '.join(v) for v in data_movimientos.values()]
    })

    # --- Mostrar las tablas ---
    display(HTML("<h3>Tabla 1: Desarrollo de la Secuencia por Pasos</h3>"))
    display(df_desarrollo)
    display(HTML("<hr><h3>Tabla 2: Lógica de Activación de Grupos</h3>"))
    display(df_activacion_grupos)
    display(HTML("<hr><h3>Tabla 3: Lógica de Activación de Movimientos</h3>"))
    display(df_activacion_movs)


# --- INSTRUCCIONES DE USO ---
# 1. Escribe tu secuencia en la variable `mi_secuencia`.
# 2. Asegúrate de no dejar espacios entre el pistón, el signo y el temporizador (ej. C+[2s]).
# 3. Ejecuta la celda.

mi_secuencia = "A- C+ C- B+ A+ B-"
analizar_secuencia_cascada(mi_secuencia)

# --- Puedes probar con otras secuencias cambiando la variable ---
# mi_secuencia = "A+ B+ B- A-"
# analizar_secuencia_cascada(mi_secuencia)

# mi_secuencia = "A+ B+ C+ C- B- A-"
# analizar_secuencia_cascada(mi_secuencia)

Análisis para la secuencia: A- C+ C- B+ A+ B-



,Paso,Movimiento,Final de Carrera Actuado,Grupo
0,1,A-,a0,Grupo 1
1,2,C+,c1,Grupo 1
2,3,C-,c0,Grupo 2
3,4,B+,b1,Grupo 2
4,5,A+,a1,Grupo 2
5,6,B-,b0,Grupo 3


,Grupo,Se Activa Con
0,Grupo 1,START y Grupo 3-b0
1,Grupo 2,Grupo 1-c1
2,Grupo 3,Grupo 2-a1


,Movimiento,Se Activa Con
0,A-,Grupo 1
1,C+,Grupo 1-a0
2,C-,Grupo 2
3,B+,Grupo 2-c0
4,A+,Grupo 2-b1
5,B-,Grupo 3


### ¿Cómo Funciona el Código?

1.  **Defines tu secuencia:** En la línea `mi_secuencia = "B- C+[2s] C- C+[2s] C- B+ A+"`, puedes poner la secuencia que quieras analizar.
2.  **División en Grupos:** El código lee tu secuencia y la divide automáticamente en grupos, siguiendo la regla del método cascada: se crea un nuevo grupo cada vez que el nombre de un pistón (A, B, C...) se repite.
3.  **Generación de Tablas:**
      * **Tabla 1 (Desarrollo):** Muestra cada paso, el movimiento que ocurre, el final de carrera que se presiona al final del movimiento y el grupo al que pertenece.
      * **Tabla 2 (Activación de Grupos):** Detalla qué señal activa el relé de cada grupo. Por ejemplo, el `Grupo 2` se activa cuando el `Grupo 1` está activo y se presiona el último final de carrera del `Grupo 1`. El `Grupo 1` siempre se inicia con el botón de `START` y la señal del último final de carrera de toda la secuencia.
      * **Tabla 3 (Activación de Movimientos):** Explica cómo se activa la electroválvula de cada movimiento. Si un movimiento es el primero de un grupo, se activa solo con la señal de ese grupo. Si no es el primero, necesita la señal del grupo Y la del final de carrera del movimiento anterior.